Установка необходимых библиотек

In [ ]:
!pip install deeppavlov
!pip install transformers
!pip install pandas

Монтирование диска Google Drive для загрузк и харанения  данных.

**ВНИМАНИЕ** для доступа к данным необходимо выполнить следующие действия:

1. Перейти по [ссылке](https://drive.google.com/drive/folders/1MnoxVXjG8o8kL0SS14E3YzbZmCovsGpR?usp=sharing)
2. Создать ярлык на данную директорию в своём Google Drive. Для этого нажмите на название директории а затем на "Добавить ярлык на Диск".

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Импортирование необходимых библиотек


In [ ]:
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

prefix = Path("/content/gdrive/My Drive/Kononov_NLP")

Выбор и загрузка модели для получения эмбеддингов

In [ ]:
bert_config = read_json(configs.embedder.bert_embedder)
# name, alias = "ru_bert_cased", "cased"
# name, alias = "ru_bert_conversational", "conversational"
name, alias = "ru_bert_sentence", "sentence"

bert_config['metadata']['variables']['BERT_PATH'] = prefix / "BERT/{}".format(name)
model = build_model(bert_config)

Проверка работы на простом примере

In [ ]:
%%time
texts = ["сделать звонок", "дозвониться"]

tokens, token_embs, subtokens, subtoken_embs, sent_max_embs, sent_mean_embs, bert_pooler_outputs = model(texts)

Подсчёт близости запросов и заголовков документов в пространстве эмбеддингов.

In [ ]:

queries_df = pd.read_csv(prefix / "queries.tsv", sep="\t", index_col=0)
titles_df = pd.read_csv(prefix / "titles.tsv", sep="\t", index_col=0).fillna("")
relation_df = pd.read_csv(prefix / "samples.tsv", sep="\t")

titles_df["text"] = titles_df["text"].apply(lambda x: " ".join(x.split()[:100]))
print(titles_df.head())


relation_df.drop(columns=["label", "sample_id"], inplace=True)
relation_df = relation_df.assign(mean_embedding_similarity=np.full(relation_df.index.size, 0.0))
relation_df = relation_df.assign(max_embedding_similarity=np.full(relation_df.index.size, 0.0))
relation_df = relation_df.assign(layer_embedding_similarity=np.full(relation_df.index.size, 0.0))
for key, group in tqdm(relation_df.groupby("query_id"), position=0, leave=True):
  doc_idx = group["doc_id"].values
  query_embedding = model([queries_df["text"].loc[key],])[-3:]
  titles = titles_df["text"].loc[doc_idx]
  print(titles.to_list())
  try:
    title_embeddings = model(titles.to_list())[-3:]
  except Exception as error:
    title_embeddings = model(titles.apply(lambda x: " ".join(x.split()[:30])).to_list())[-3:]

  relation_df.loc[group.index, "mean_embedding_similarity"] = cosine_similarity(query_embedding[0].reshape(1, -1),
                                                                          title_embeddings[0]).reshape(-1)
  relation_df.loc[group.index, "max_embedding_similarity"] = cosine_similarity(query_embedding[1].reshape(1, -1),
                                                                          title_embeddings[1]).reshape(-1)
  relation_df.loc[group.index, "layer_embedding_similarity"] = cosine_similarity(query_embedding[2].reshape(1, -1),
                                                                          title_embeddings[2]).reshape(-1)
  



path = prefix / "semantic/bert_{}_query_doc_embedding_similarity.tsv".format(alias)
relation_df.to_csv(path, sep="\t", index=False)
print("Result saved to file: {}".format(path))